# CEAI 1110 Deep Learning in Action: coding action #4

Coding activity 4 contains 2 parts: Build LSTM model to classify whether a sentence is a insincere one or not

`(1) Preprocessing for QA text data
(2) Build LSTM model for sequence learning `


`please kindly refer your coding activity sheet #4 for correspondence`

Coach: Yimin Nie

email: ymnie888@gmail.com

In [ ]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import time
import re
import gc
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM,CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D,GlobalAveragePooling2D,SpatialDropout1D
from keras.layers import Bidirectional, GlobalMaxPool1D,concatenate,Concatenate,GlobalAvgPool1D,BatchNormalization,Flatten
from keras.layers import GlobalAveragePooling1D,GlobalMaxPooling1D,PReLU,Reshape,Conv2D,MaxPool2D,add,SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras.callbacks import Callback
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint

`Code 1.1: read data and perform text preprocessing`

In [ ]:
train = pd.read_csv("train.zip",compression='zip')
print(train.shape)
train.head()

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

In [ ]:
embed_size = 300 # how big is each word vector
max_features = 95000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

In [ ]:
# Clean the text
train["question_text"] = train["question_text"].apply(lambda x: clean_text(x))

In [ ]:
## split to train and validation
train, valid = train_test_split(train, test_size=0.1, random_state=2018)

In [ ]:
## fill up the missing values
train_X = train["question_text"].fillna("unknown").apply(lambda x:x.lower()).values
val_X = valid["question_text"].fillna("unknown").apply(lambda x:x.lower()).values


## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)

## Get the target values
train_y = train['target'].values
val_y = valid['target'].values
print(train_X.shape,val_X.shape)

#shuffling the data
seed = np.random.randint(0, 99999)
trn_idx = np.random.permutation(len(train_X))
val_idx = np.random.permutation(len(val_X))

train_X = train_X[trn_idx]
val_X = val_X[val_idx]
train_y = train_y[trn_idx]
val_y = val_y[val_idx]

`Code 1.2: use LSTM with bidirecional connection to build classification model`

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-3), metrics=['accuracy'])  # use accuracy as metrics

In [ ]:
callback = [EarlyStopping(monitor='val_loss',patience=1),
            ReduceLROnPlateau(patience=1, verbose=0),
            ModelCheckpoint('model.h5',save_best_only=True,verbose=0),
            ]
model.fit(x=train_X, y=train_y, 
          batch_size=512, 
          epochs=2, 
          validation_data=(val_X, val_y),
          callbacks=callback,
          verbose=1
         )

`Code 1.3: use F1 score as evaluation`

Please understand how to implement f1 score use keras

In [ ]:
def f1(y_true, y_pred):
    '''
    implement of f1 score in keras
    '''
    def recall(y_true, y_pred):
        """
        Recall metric.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """
        Precision metric.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-3), metrics=[f1])  # use f1 as metrics

In [ ]:
callback = [EarlyStopping(monitor='val_loss',patience=1),
            ReduceLROnPlateau(patience=1, verbose=0),
            ModelCheckpoint('model.h5',save_best_only=True,verbose=0),
            ]
model.fit(x=train_X, y=train_y, 
          batch_size=512, 
          epochs=5, 
          validation_data=(val_X, val_y),
          callbacks=callback,
          verbose=1
         )